In [5]:
#os library to interact with the os
import os
#pickle library to store and read large datasets quickly
#converts from python objects to bytekey and then back to a python object
import pickle
#numpy for linear algebra operations
import numpy as np
#pandas for various dataframes and operations
import pandas as pd
#periodic table for attributes of elements
from pymatgen.core.periodic_table import Element

In [6]:
#This cell reads from a excel file the very first time the program is run. 
#In each following run, it reads from a pickle file, which allows it to read faster.


#check if the file 'df.pkl' is in the current directory
if not os.path.exists('df.pkl'):
    #if it is not in the current directory, then read the contents of 'jz7b02203_si_002.xlsx' into a pandas dataframe
    raw_df = pd.read_excel('jz7b02203_si_002.xlsx')
    #Drop all the NA values from the pandas dataframe
    df = raw_df[['System', 'LowGap_SOHSE_Approx']][raw_df['LowGap_SOHSE_Approx'] >= 0].dropna()
    #store the pandas dataframe as a pickle file (stored as bytekey)
    df.to_pickle('df.pkl')
else:
    #if the file does exist, read the pickle file into a pandas dataframe
    df = pd.read_pickle('df.pkl')

In [7]:
#read the file 'ZungerValenceRadius.csv' into a pandas dataframe
_ = pd.read_csv('ZungerValenceRadius.csv')
# Create dictionaries for each radius (Rs, Rp, Rd)
#Gets the element and Rs value from the pandas dataframe and converts it to a dictionary
RS_DICT = pd.Series(_['Rs'].values,index=_['Element']).to_dict()
#Gets the element and Rp value from the pandas dataframe and converts it to a dictionary
RP_DICT = pd.Series(_['Rp'].values,index=_['Element']).to_dict()
#Gets the element and Rd value from the pandas dataframe and converts it to a dictionary
RD_DICT = pd.Series(_['Rd'].values,index=_['Element']).to_dict()


In [8]:
//
_ = pd.read_csv('H_and_L_and_R_squared.csv')

H_DICT = pd.Series(_['H'].values, index=_['Element']).to_dict()
H_DICT['MA'] = -0.63090
H_DICT['FA'] = -0.51978

L_DICT = pd.Series(_['L'].values, index=_['Element']).to_dict()
L_DICT['MA'] = -0.21720
L_DICT['FA'] = -0.24230

R_SQUARED_DICT = pd.Series(_['R_squared'].values, index=_['Element']).to_dict()
R_SQUARED_DICT['MA'] = 94.7121
R_SQUARED_DICT['FA'] = 158.5805

In [9]:
_ = pd.read_csv('ion.csv', sep=';', header=None,
                names=['AtomicNumber', 'Element',
                       'IP1', 'IP2', 'IP3'])

IONIC_POT_DICT = pd.Series(_['IP1'].values, index=_['Element']).to_dict()

In [10]:
IONIC_R_DICT = {Element.from_Z(i).symbol: Element.from_Z(i).average_ionic_radius
                for i in range(1, 92)}

# Effective ionic radius values from Kieslich et al.
# [Chem. Sci., 2014, 5, 4712]
IONIC_R_DICT['MA'] = 2.17
IONIC_R_DICT['FA'] = 2.53

# Use pymatgen to get electronegativity and store it in a dictionary
ENEG_DICT = {Element.from_Z(i).symbol: Element.from_Z(i).X
             for i in range(1, 92)}

# Use pymatgen to get ionic radii and store it into a dictionary
BOIL_DICT = {Element.from_Z(i).symbol: Element.from_Z(i).boiling_point
             if Element.from_Z(i).boiling_point else 0 for i in range(1, 92)}

# Use pymatgen to get melting points and store it into a dictionary
MELT_DICT = {Element.from_Z(i).symbol: Element.from_Z(i).melting_point / 1000
             if Element.from_Z(i).melting_point !='maybe about 300 K' else
             300 for i in range(1, 92)}

C:\Miniconda3\lib\site-packages\pymatgen\core\periodic_table.py:427: UserWarning: No electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  % self.symbol)
C:\Miniconda3\lib\site-packages\pymatgen\core\periodic_table.py:427: UserWarning: No electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  % self.symbol)
C:\Miniconda3\lib\site-packages\pymatgen\core\periodic_table.py:427: UserWarning: No electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  % self.symbol)


In [11]:
#del magpie_features
mpf = pd.read_csv('output.csv', header=None,
                  names=sorted(os.listdir('t/')))

mpf[mpf.columns[1:]] = mpf[mpf.columns[1:]].apply(pd.to_numeric, errors='coerce')
mpf.drop(['n_ws^third', 'phi'], axis=1, inplace=True)

MPF_DICTS = {i: pd.Series(mpf[i].values, index=mpf['AAAAA']).to_dict()
             for i in mpf.columns[1:]}

FileNotFoundError: [WinError 3] The system cannot find the path specified: 't/'

In [25]:
# Parse A sites from the chemical formula and store it in a Series
a_sites = df['System'].str.extract(r'(C4N4H24|C4N8H20|Cs4)')
a_sites.replace('C4N4H24', 'MA', inplace=True)
a_sites.replace('C4N8H20', 'FA', inplace=True)
a_sites.replace('Cs4', 'Cs', inplace=True)

# Store a_sites a column in df
df['A_sites'] = a_sites

# Parse B sites from chemical formula
# Separate into two DataFrames:
# B != B' (true_dbls), B == B' (false_dbls)
true_dbls = df[df['System'].str.contains(r'[a-zA-Z]+(?=2[a-zA-Z]+2)')].copy()
b_sites = true_dbls['System'].str.extract(r'(?<=H24|H20|Cs4)([a-zA-Z]+)(?=2[a-zA-Z]+2)')
bp_sites = true_dbls['System'].str.extract(r'(?<=[a-zA-Z]2)([a-zA-Z]+)(?=2)')
true_dbls['B_sites'] = b_sites
true_dbls['B_prime_sites'] = bp_sites

false_dbls = df[~df['System'].str.contains(r'[a-zA-Z]+(?=2[a-zA-Z]+2)')].copy()
b_sites = false_dbls['System'].str.extract(r'(?<=H24|H20|Cs4)([a-zA-Z]+)(?=4)')
false_dbls['B_sites'] = b_sites
false_dbls['B_prime_sites'] = b_sites

# Put these values together back into df
df = pd.concat([true_dbls, false_dbls])
df.sort_values('System', inplace=True)
df.reset_index(drop=True, inplace=True)

# Parse A sites from the chemical formula and store it in a Series
x_sites = df['System'].str.extract(r'(Cl|Br|I)(?=12)')
# Store x_sites a column in df
df['X_sites'] = x_sites
false_dbls

,System,LowGap_SOHSE_Approx,A_sites,B_sites,B_prime_sites,X_sites,A_IonicRadius,X_IonicRadius,TolFactor_sum,TolFactor_dif,...,sqrt_A_L,square_A_L,cube_A_L,log_one_plus_A_L,exp_A_L,sqrt_A_R_SQUARED,square_A_R_SQUARED,cube_A_R_SQUARED,log_one_plus_A_R_SQUARED,exp_A_R_SQUARED
58,C4N4H24Al4Br12,3.184055,MA,Al,Al,Br,0.5,0.20777,0.416835,0.0,...,0.371467,0.019041,0.002627,0.129262,1.147962,0.435013,0.03581,0.006777,0.173311,1.208326
59,C4N4H24Al4Cl12,3.836635,MA,Al,Al,Cl,0.5,0.00000,0.452524,0.0,...,0.371467,0.019041,0.002627,0.129262,1.147962,0.435013,0.03581,0.006777,0.173311,1.208326
60,C4N4H24Al4I12,2.038116,MA,Al,Al,I,0.5,1.00000,0.315212,0.0,...,0.371467,0.019041,0.002627,0.129262,1.147962,0.435013,0.03581,0.006777,0.173311,1.208326
75,C4N4H24Au4Br12,1.388320,MA,Au,Au,Br,0.5,0.20777,0.207576,0.0,...,0.371467,0.019041,0.002627,0.129262,1.147962,0.435013,0.03581,0.006777,0.173311,1.208326
76,C4N4H24Au4Cl12,1.664521,MA,Au,Au,Cl,0.5,0.00000,0.224050,0.0,...,0.371467,0.019041,0.002627,0.129262,1.147962,0.435013,0.03581,0.006777,0.173311,1.208326
77,C4N4H24Au4I12,1.342278,MA,Au,Au,I,0.5,1.00000,0.157984,0.0,...,0.371467,0.019041,0.002627,0.129262,1.147962,0.435013,0.03581,0.006777,0.173311,1.208326
148,C4N4H24B4Br12,4.541533,MA,B,B,Br,0.5,0.20777,0.628913,0.0,...,0.371467,0.019041,0.002627,0.129262,1.147962,0.435013,0.03581,0.006777,0.173311,1.208326
149,C4N4H24B4Cl12,5.410696,MA,B,B,Cl,0.5,0.00000,0.690815,0.0,...,0.371467,0.019041,0.002627,0.129262,1.147962,0.435013,0.03581,0.006777,0.173311,1.208326
150,C4N4H24B4I12,3.179699,MA,B,B,I,0.5,1.00000,0.462052,0.0,...,0.371467,0.019041,0.002627,0.129262,1.147962,0.435013,0.03581,0.006777,0.173311,1.208326
255,C4N4H24C4Br12,2.695498,MA,C,C,Br,0.5,0.20777,0.744862,0.0,...,0.371467,0.019041,0.002627,0.129262,1.147962,0.435013,0.03581,0.006777,0.173311,1.208326


In [13]:
def get_A_quant(data_dict, title=''):
    '''
    --------------------------------------------
    Get the desired feature for the A site.
    --------------------------------------------
    Inputs:  data_dict (dict)
             Contains atomic symbols (e.g. 'Li')
             as keys and the desired property
             as values.

             title (str, default '')
             The title of the column to be
             placed into df.
    --------------------------------------------
    Outputs: No outputs.
             Saves the A quantity as a column
             of title title into the global
             variable df.
    --------------------------------------------
    '''
    if not 'A_' + title in df:
        print(f'Adding {title} of A to df...')
    else:
        print(f'Overriding {title} of A to df (CAUTION!)')
    quant = df['A_sites'].replace(data_dict) + 0.0
    df['A_' + title] = quant.astype(np.float32)
    print('>>> Done!')

def get_B_quant(data_dict, title=''):
    '''
    --------------------------------------------
    Get the desired feature for the B site.
    --------------------------------------------
    Inputs:  data_dict (dict)
             Contains atomic symbols (e.g. 'Li')
             as keys and the desired property
             as values.

             title (str, default '')
             The title of the column to be
             placed into df.
    --------------------------------------------
    Outputs: No outputs.
             Saves the sum and abs(difference)
             of the desired feature for atoms
             B and B' of titles
             'B_' + title + '_sum', and
             'B_' + title + '_dif'
    --------------------------------------------
    '''
    if not 'B_' + title + '_sum' in df:
        print(f'Adding {title} of B to df...')
    else:
        print(f'Overriding {title} of B to df (CAUTION!)')
    summ = df['B_sites'].replace(data_dict) + df['B_prime_sites'].replace(data_dict)
    diff = (np.abs(df['B_sites'].replace(data_dict) -
            df['B_prime_sites'].replace(data_dict)))
    df['B_' + title + '_sum'] = summ.astype(np.float32)
    df['B_' + title + '_dif'] = diff.astype(np.float32)
    print('>>> Done!')
    
def get_X_quant(data_dict, title=''):
    if not 'X_' + title in df:
        print(f'Adding {title} of X to df...')
    else:
        print(f'Overriding {title} of X to df (CAUTION!)')
    quant = df['X_sites'].replace(data_dict) + 0.
    df['X_' + title] = quant.astype(np.float32)
    print('>>> Done!')

In [14]:
def tolerance_factor(Ra, Rb, Rx):
    return (Ra + Rx) / (np.sqrt(2) * (Rb + Rx))

def get_tolerance_factor(title='TolFactor'):
    '''
    --------------------------------------------
    Get the tolerance factor for perovskites.
    --------------------------------------------
    Inputs:  title (str, default 'TolFactor')
             The title of the column to be
             placed into df.
             Rb (float)
    --------------------------------------------
    Outputs: No outputs.
             Saves the sum and abs(difference)
             of the toelrance factor for atoms
             B and B' of titles
             title + '_sum', and
             title + '_dif'
    --------------------------------------------
    '''
    if not 'TolFactor_sum' in df:
        print(f'Adding {title} to df...')
    else:
        print(f'Overriding {title} to df (CAUTION!)')

    print('Calling get_A_quant(IONIC_R_DICT, \'IonicRadius\')')
    get_A_quant(IONIC_R_DICT, 'IonicRadius')

    print('Calling get_X_quant(IONIC_R_DICT, \'IonicRadius\')')
    get_X_quant(IONIC_R_DICT, 'IonicRadius')

    df['Rb'] = df['B_sites'].replace(IONIC_R_DICT)
    df['Rbp'] = df['B_prime_sites'].replace(IONIC_R_DICT)

    df[title + '_sum'] = (
        tolerance_factor(df['A_IonicRadius'],
                         df['Rb'], df['X_IonicRadius'])
        + tolerance_factor(df['A_IonicRadius'],
                           df['Rbp'], df['X_IonicRadius'])
    ).astype(np.float32)

    df[title + '_dif'] = np.abs(
        tolerance_factor(df['A_IonicRadius'],
                         df['Rb'], df['X_IonicRadius'])
        - tolerance_factor(df['A_IonicRadius'],
                           df['Rbp'], df['X_IonicRadius'])
    ).astype(np.float32)

    df.drop(['Rb', 'Rbp'], axis=1, inplace=True)
    print('>>> Done with TolFactor!')

get_tolerance_factor()

Adding TolFactor to df...
Calling get_A_quant(IONIC_R_DICT, 'IonicRadius')
Adding IonicRadius of A to df...
>>> Done!
Calling get_X_quant(IONIC_R_DICT, 'IonicRadius')
Adding IonicRadius of X to df...
>>> Done!
>>> Done with TolFactor!


In [15]:
names = [(RS_DICT, 'Rs'),
         (RP_DICT, 'Rp'),
         (RD_DICT, 'Rd'),
         (IONIC_R_DICT, 'IonicRadius'),
         (IONIC_POT_DICT, 'FirstIonizationPotential'),
         (ENEG_DICT, 'Electronegativity'),
         (BOIL_DICT, 'BoilingPoint'),
         (H_DICT, 'H'),
         (L_DICT, 'L'),
         (R_SQUARED_DICT, 'R_SQUARED'),
         (MELT_DICT, 'MeltingPoint')]

# Fetch B and X quantities
for pair in names:
    get_B_quant(*pair)
for pair in names:
    if pair != (IONIC_R_DICT, 'IonicRadius'):
        get_X_quant(*pair)
        
for pair in [(H_DICT, 'H'), (L_DICT, 'L'), (R_SQUARED_DICT, 'R_SQUARED')]:
    get_A_quant(*pair)
    
for k in MPF_DICTS:
    get_B_quant(MPF_DICTS[k], k)
    get_X_quant(MPF_DICTS[k], k)

Adding Rs of B to df...
>>> Done!
Adding Rp of B to df...
>>> Done!
Adding Rd of B to df...
>>> Done!
Adding IonicRadius of B to df...
>>> Done!
Adding FirstIonizationPotential of B to df...
>>> Done!
Adding Electronegativity of B to df...
>>> Done!
Adding BoilingPoint of B to df...
>>> Done!
Adding H of B to df...
>>> Done!
Adding L of B to df...
>>> Done!
Adding R_SQUARED of B to df...
>>> Done!
Adding MeltingPoint of B to df...
>>> Done!
Adding Rs of X to df...
>>> Done!
Adding Rp of X to df...
>>> Done!
Adding Rd of X to df...
>>> Done!
Adding FirstIonizationPotential of X to df...
>>> Done!
Adding Electronegativity of X to df...
>>> Done!
Adding BoilingPoint of X to df...
>>> Done!
Adding H of X to df...
>>> Done!
Adding L of X to df...
>>> Done!
Adding R_SQUARED of X to df...
>>> Done!
Adding MeltingPoint of X to df...
>>> Done!
Adding H of A to df...
>>> Done!
Adding L of A to df...
>>> Done!
Adding R_SQUARED of A to df...
>>> Done!


NameError: name 'MPF_DICTS' is not defined

In [16]:
df_numeric = df[df.columns[6:]]

In [17]:
df[df.columns[6:]] = (df_numeric - df_numeric.min()) / (df_numeric.max() - df_numeric.min())

In [18]:
for col, val in zip(df.columns, df.isna().any()):
    if val:
        df.drop(col, axis=1, inplace=True)

ORIG_FEATURES = df.columns[6:]

In [19]:
for feature in ORIG_FEATURES:
    df['sqrt_' + feature] = np.sqrt(df[feature])
    df['square_' + feature] = df[feature] ** 2
    df['cube_' + feature] = df[feature] ** 3
    df['log_one_plus_' + feature] = np.log(1 + df[feature])
    df['exp_' + feature] = np.exp(df[feature])
    print(f'>>> Done with {feature}!')

# Save raw values into a file 'raw_values.pkl'
with open('raw_values.pkl','wb') as f:
    pickle.dump(df[df.columns[6:]].values, f)
    
with open('for_y.pkl', 'wb') as f:
    pickle.dump(df[['System', 'LowGap_SOHSE_Approx']], f)
    
df.to_pickle('entire_df.pkl')

>>> Done with A_IonicRadius!
>>> Done with X_IonicRadius!
>>> Done with TolFactor_sum!
>>> Done with TolFactor_dif!
>>> Done with B_Rs_sum!
>>> Done with B_Rs_dif!
>>> Done with B_Rp_sum!
>>> Done with B_Rp_dif!
>>> Done with B_Rd_sum!
>>> Done with B_Rd_dif!
>>> Done with B_IonicRadius_sum!
>>> Done with B_IonicRadius_dif!
>>> Done with B_FirstIonizationPotential_sum!
>>> Done with B_FirstIonizationPotential_dif!
>>> Done with B_Electronegativity_sum!
>>> Done with B_Electronegativity_dif!
>>> Done with B_BoilingPoint_sum!
>>> Done with B_BoilingPoint_dif!
>>> Done with B_H_sum!
>>> Done with B_H_dif!
>>> Done with B_L_sum!
>>> Done with B_L_dif!
>>> Done with B_R_SQUARED_sum!
>>> Done with B_R_SQUARED_dif!
>>> Done with B_MeltingPoint_sum!
>>> Done with B_MeltingPoint_dif!
>>> Done with X_Rs!
>>> Done with X_Rp!
>>> Done with X_Rd!
>>> Done with X_FirstIonizationPotential!
>>> Done with X_Electronegativity!
>>> Done with X_BoilingPoint!
>>> Done with X_H!
>>> Done with X_L!
>>> Done 

In [20]:
with open('column_names.pkl','wb') as f:
    pickle.dump(list(df.columns[6:]), f)

In [21]:
with open('column_names.pkl', 'rb') as f:
    cols = pickle.load(f)

In [22]:
cols

['A_IonicRadius',
 'X_IonicRadius',
 'TolFactor_sum',
 'TolFactor_dif',
 'B_Rs_sum',
 'B_Rs_dif',
 'B_Rp_sum',
 'B_Rp_dif',
 'B_Rd_sum',
 'B_Rd_dif',
 'B_IonicRadius_sum',
 'B_IonicRadius_dif',
 'B_FirstIonizationPotential_sum',
 'B_FirstIonizationPotential_dif',
 'B_Electronegativity_sum',
 'B_Electronegativity_dif',
 'B_BoilingPoint_sum',
 'B_BoilingPoint_dif',
 'B_H_sum',
 'B_H_dif',
 'B_L_sum',
 'B_L_dif',
 'B_R_SQUARED_sum',
 'B_R_SQUARED_dif',
 'B_MeltingPoint_sum',
 'B_MeltingPoint_dif',
 'X_Rs',
 'X_Rp',
 'X_Rd',
 'X_FirstIonizationPotential',
 'X_Electronegativity',
 'X_BoilingPoint',
 'X_H',
 'X_L',
 'X_R_SQUARED',
 'X_MeltingPoint',
 'A_H',
 'A_L',
 'A_R_SQUARED',
 'sqrt_A_IonicRadius',
 'square_A_IonicRadius',
 'cube_A_IonicRadius',
 'log_one_plus_A_IonicRadius',
 'exp_A_IonicRadius',
 'sqrt_X_IonicRadius',
 'square_X_IonicRadius',
 'cube_X_IonicRadius',
 'log_one_plus_X_IonicRadius',
 'exp_X_IonicRadius',
 'sqrt_TolFactor_sum',
 'square_TolFactor_sum',
 'cube_TolFactor_sum

In [23]:
ENEG_DICT

{'H': 2.2,
 'He': nan,
 'Li': 0.98,
 'Be': 1.57,
 'B': 2.04,
 'C': 2.55,
 'N': 3.04,
 'O': 3.44,
 'F': 3.98,
 'Ne': nan,
 'Na': 0.93,
 'Mg': 1.31,
 'Al': 1.61,
 'Si': 1.9,
 'P': 2.19,
 'S': 2.58,
 'Cl': 3.16,
 'Ar': nan,
 'K': 0.82,
 'Ca': 1.0,
 'Sc': 1.36,
 'Ti': 1.54,
 'V': 1.63,
 'Cr': 1.66,
 'Mn': 1.55,
 'Fe': 1.83,
 'Co': 1.88,
 'Ni': 1.91,
 'Cu': 1.9,
 'Zn': 1.65,
 'Ga': 1.81,
 'Ge': 2.01,
 'As': 2.18,
 'Se': 2.55,
 'Br': 2.96,
 'Kr': 3.0,
 'Rb': 0.82,
 'Sr': 0.95,
 'Y': 1.22,
 'Zr': 1.33,
 'Nb': 1.6,
 'Mo': 2.16,
 'Tc': 1.9,
 'Ru': 2.2,
 'Rh': 2.28,
 'Pd': 2.2,
 'Ag': 1.93,
 'Cd': 1.69,
 'In': 1.78,
 'Sn': 1.96,
 'Sb': 2.05,
 'Te': 2.1,
 'I': 2.66,
 'Xe': 2.6,
 'Cs': 0.79,
 'Ba': 0.89,
 'La': 1.1,
 'Ce': 1.12,
 'Pr': 1.13,
 'Nd': 1.14,
 'Pm': 1.13,
 'Sm': 1.17,
 'Eu': 1.2,
 'Gd': 1.2,
 'Tb': 1.1,
 'Dy': 1.22,
 'Ho': 1.23,
 'Er': 1.24,
 'Tm': 1.25,
 'Yb': 1.1,
 'Lu': 1.27,
 'Hf': 1.3,
 'Ta': 1.5,
 'W': 2.36,
 'Re': 1.9,
 'Os': 2.2,
 'Ir': 2.2,
 'Pt': 2.28,
 'Au': 2.54,
 'Hg': 2.0